In [88]:
# Generate code (have server running)

import webbrowser
from urllib.parse import urlencode, urljoin

queryparams = {
    "client_id":"c902e10b-a3a7-43f1-9752-455b331f1d83",
    "redirect_uri":"http://localhost:8080",
    "response_type":"code"
}


webbrowser.open(
    urljoin('https://api.upstox.com/v2/login/authorization/dialog', '?' + urlencode(queryparams))
)

True

In [89]:
# Get auth token

code = 'Xsy9EE'

In [90]:
import requests, json

headers = {
    'Content-Type' : 'application/x-www-form-urlencoded',
    'Accept' : 'application/json'
}

data = {
    'code' : code,
    'client_id' : 'c902e10b-a3a7-43f1-9752-455b331f1d83',
    'client_secret' : 'axvei02r3f',
    'redirect_uri' : 'http://localhost:8080',
    'grant_type' : 'authorization_code'
}

response = requests.request(
    method = 'POST',
    url = 'https://api.upstox.com/v2/login/authorization/token',
    headers = headers,
    data = data
)

# print(response.json())
print(response.status_code)

auth_token = response.json()['access_token']

200


In [87]:
# generate and use websocket URL
import websocket

headers = {
    'Authorization' : f'Bearer {auth_token}',
    'Accept' : 'application/json'
}

response = requests.get(
    url = 'https://api.upstox.com/v2/feed/market-data-feed/authorize',
    headers = headers
)

print(f'URL generation respone -> {response.status_code}')

data = {
            "guid": "someguid",
            "method": "sub",
            "data": {
                "mode": "full",
                "instrumentKeys": ["NSE_INDEX|Nifty Bank", "NSE_INDEX|Nifty 50"]
            }
        }

# Define WebSocket event handlers
def on_message(ws, message):
    print(f"Received: {message}")

def on_error(ws, error):
    print(f"Error: {error}")

def on_close(ws, close_status_code, close_reason):
    print("Connection closed")

def on_open(ws):
    print("Connection opened")
    ws.send(json.dumps(data).encode('utf-8'))


# Connect to WebSocket
ws_url = response.json()['data']['authorized_redirect_uri']  # Replace with your WebSocket URL
ws = websocket.WebSocketApp(
    ws_url,
    header=headers,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# Run the WebSocket connection
ws.run_forever()


URL generation respone -> 200
Connection opened
Error: 
Connection closed


True

In [98]:
# current market quote

headers = {
    'Authorization' : f'Bearer {auth_token}',
    'Accept' : 'application/json'
}

queryparams = {
    "instrument_key":"NSE_EQ|INE848E01016",
    "interval":"I30"
}

response = requests.get(
    url = urljoin('https://api.upstox.com/v2/market-quote/ohlc', '?' + urlencode(queryparams)),
    headers = headers
)

response.json()

{'status': 'success',
 'data': {'NSE_EQ:NHPC': {'ohlc': {'open': 91.05,
    'high': 91.18,
    'low': 91.0,
    'close': 91.02},
   'last_price': 90.81,
   'instrument_token': 'NSE_EQ|INE848E01016'}}}

In [121]:
# Historic OHLC data

url = "https://api.upstox.com/v2/historical-candle/{instrument_key}/{interval}/{to_date}/{from_date}"

formatted_url = url.format(
    instrument_key = 'NSE_EQ|INE848E01016',
    interval = 'day', # 1minute, 30minute, day, week, month
    to_date = '2024-10-10',
    from_date = '2001-10-10'
)

headers = {
    #'Authorization' : f'Bearer {auth_token}',
    'Accept' : 'application/json'
}

response = requests.get(
    url = formatted_url,
    headers = headers
)

response.json()['data']['candles'][len(response.json()['data']['candles'])-1]

['2009-09-01T00:00:00+05:30', 42, 42, 36.65, 36.75, 508832326, 0]